In [ ]:
# !pip install scipy
# !pip install pysindy plotly

In [ ]:
import numpy as np
import plotly.graph_objs as go
from scipy.integrate import solve_ivp
import pysindy as ps

# Параметры системы Лоренца
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0

# Уравнения Лоренца
def lorenz(t, state):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Начальные условия и время интегрирования
initial_state = [1.0, 1.0, 1.0]
t_span = (0, 25)  # временной интервал
t_eval = np.linspace(t_span[0], t_span[1], 1000)  # временные точки

# Интеграция системы
sol = solve_ivp(lorenz, t_span, initial_state, t_eval=t_eval)
x = sol.y[0]
y = sol.y[1]
z = sol.y[2]

# Стек данных (X)
data = np.vstack((x, y, z)).T

# Задержка времени (шаг)
dt = t_eval[1] - t_eval[0]

# Обучение модели SINDy
model = ps.SINDy()
model.fit(data, t=dt)
model.print()

# Прогнозирование и визуализация
simulated_data = model.simulate(initial_state, t_eval)

# Построение интерактивного графика с Plotly
fig = go.Figure()

# Исходные данные
fig.add_trace(go.Scatter3d(
    x=data[:, 0], y=data[:, 1], z=data[:, 2],
    mode='lines', name='Original',
    line=dict(color='blue', width=2)
))

# Симулированные данные
fig.add_trace(go.Scatter3d(
    x=simulated_data[:, 0], y=simulated_data[:, 1], z=simulated_data[:, 2],
    mode='lines', name='Simulated',
    line=dict(color='red', width=2)
))

# Настройки графика
fig.update_layout(
    title='Lorenz System Reconstruction using SINDy',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    width=800,
    height=600
)

fig.show()
